# Multilayer Perceptron with Dense Layers

### Imports

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
from matplotlib import pyplot

### Default values

In [2]:
DEFAULT_EPOCH = 50
DEFAULT_BATCH = 72

VALIDATION_SPLIT = 0.3
TESTING_SPLIT = 0.3

TOTAL = 744
N_TRAIN = 550

We set a default batch size value and number of epochs. Also, the training and testing datasets have a fixed size. The former is defined by the N_TRAIN variable, whilst the latter is TOTAL-N_TRAIN

VALIDATION_SPLIT and TESTING_SPLIT denote respectively the percentage of training that is used during the validation step and the percentage of inputs that is used during the evaluation step

### Keras Dense MLP Class

The data processor and dataset are respectively defined as follows:

In [6]:
self.processor = processor #The data processor is responsible for handling operations with the dataset such as scaling, rescaling and preparing the data from a time series table to a supervised learning approach
self.values = values #The values variable is the result of the values from the csv file read by the Pandas package

The data processor is responsible for handling operations with the dataset such as scaling, rescaling and preparing the data from a time series table to a supervised learning approach

The values variable is the result of the values from the csv file read by the Pandas package

In [ ]:
self.hours = args.hours if args.hours else 1
self.checkpoint = args.checkpoint if args.checkpoint else "dense_" + len(args.neurons) + "_layers_checkpoint.keras"
self.epochs = args.epochs if args.epochs else DEFAULT_EPOCH
self.batch  = args.batch if args.batch else DEFAULT_BATCH

It is possible to set some variable using the arguments when calling the main.py from the command-line, such as the number of hours to consider for prediction (e.g. 1, 2 ou 3 hours into the future), the number of epochs, the batch size and others.

The checkpoint variable represents the filename that contains the weights obtained during training.

##### Training and Testing datasets

In [ ]:
self.build_data()

def build_data(self):
    inputs = self.values[0:N_INPUTS]
    prediction = self.values[N_INPUTS:TOTAL]
    
    np.random.shuffle(inputs)
    N_TEST = int(N_INPUTS*TESTING_SPLIT)

    self.train_X, self.train_y = inputs[:(N_INPUTS-N_TEST), :-1], inputs[:(N_INPUTS-N_TEST), -1]
    self.test_X, self.test_y = inputs[(N_INPUTS-N_TEST):, :-1], inputs[(N_INPUTS-N_TEST):, -1]

    self.prediction_X, self.prediction_y = prediction[:, :-1], prediction[:, -1]

We call the build_data() method in the class constructor. This method separates the self.values variable into training and testing arrays according to the number of training and testing samples specified previously.

Considering the prediction of a single attribute within the next hour, the training and testing arrays have n+1 elements, in which n is the number of inputs. Considering now an ANN with 9 inputs and 1 output, the inputs for the ANN are represented from the 1st to the 9th element, whilst the 10th element is the next hour value to be predicted. Therefore, we set a self.train_x as the inputs (from the 1st to the 9th element) to be used during training and self.train_y as the labels (the 10th element) to be used during training. We apply the same logic for the testing array.

#### Model and Layers

In [ ]:
self.model = keras.Sequential()

self.model.add(keras.layers.Dense(
      units=args.neurons[0],
      activation="tanh",
      input_shape=(args.input,)
))

for key, neurons in enumerate(args.neurons):
    if key < len(args.neurons) - 1:
        self.model.add(keras.layers.Dense(units=args.neurons[key+1], activation="tanh"))
    else:
        self.model.add(keras.layers.Dense(units=args.output, activation="tanh"))

self.model.add(keras.layers.Dense(units=args.output))

We define our Keras model as a Sequential from the keras package. Then we define the ann's layers. It is mandatory to have, at least, three layers, in which: 1 is the input layer, 1 is the hidden layer and 1 is the output layer.

The activation function for all layers, except the output layer, is the hyperbolic tangent. The output layer uses a linear activation function.

The number of hidden layers and, consequently, the number of neurons for each hidden layer are determined by the flag -n (--neurons). For example, if we define *-n 6 5* the ANN ought to have 2 hidden layers containing respectively 6 and 5 neurons each.

The layers are defined as the Keras' Dense layer. The shape of the first layer input is defined as a (n_inputs,) tuple. Also this layer produces as many outputs as there are neurons for the next hidden layer, i.e., in the previous example (-n 6 5) each node of the input layer produces 6 outputs (each one connected to the next layer's neuron). The same logic applies for the each hidden layer, except the last one. That is: each node of the first hidden layer ought to produce 5 outputs. If our example had more hidden layers the 2nd hidden layer should produce as output the number of neurons belonging to a third hidden layer. However, since the 2nd hidden layer is the last one before the output layer, it produces a number of outputs according to the number of neurons for the output layer. The last layer is the output layer and it produces a number of n_outputs output(s).

#### Training method

In [ ]:
def train(self):
    print("train")
    checkpoint = keras.callbacks.ModelCheckpoint(
      filepath=self.checkpoint,
      monitor="loss",
      verbose=1,
      save_weights_only=True,
      save_best_only=True
    )
    early_stopping = keras.callbacks.EarlyStopping(monitor="loss", patience=5, verbose=1)
    tensorboard = keras.callbacks.TensorBoard(log_dir="./logs/", histogram_freq=0, write_graph=False)
    
    callbacks = [
      checkpoint, early_stopping, tensorboard
    ]
    
    history = self.model.fit(
      self.train_X, self.train_y, epochs=self.epochs,
      steps_per_epoch=72,
      validation_steps=10,
      validation_split=VALIDATION_SPLIT,
      callbacks=callbacks
    )

    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='validation')
    pyplot.legend()
    pyplot.show()

We define three callbacks functions before training the model.

The checkpoint callback is responsible for dinamically saving the weights during the training step as soon as a improvement in the loss value (the variable being monitored) is detected. The filepath is defined by the self.checkpoint variable presented previously.

The early stopping checkpoint also monitors the loss value during training and it is meant to stop the training step as soon as no improvement is detected in the monitored value.

The tensorboard checkpoint generates logs of events to be visualised using Tensorboard.

Finally, the model is trained using the fit() method. We supply self.train_X and self.train_y as respectively the training data and the training label data. The number of epochs are determined by self.epochs and can either be the default value (50 epochs) or user-defined through the -e flag. Each epoch takes 36 steps to finish. Currently the whole training data and training label data are being used for validation (see validation_split=1).

It is important to say that the label array contains only values for prediction within the next hour, that is: assuming that the 1th element of the training array references contents of the 13th hour of any given date, the 1th element of the label array will, therefore, contain an attribute referencing the 14th hour of the same day.

At the end of the training the graphic containing both the loss and validation loss values is plotted. We can use this graphic to investigate whether the training overfitted or not.

#### Evaluation method

In [ ]:
def evaluate(self):
    result = self.model.evaluate(x=self.train_X, y=self.train_y)

    for res, metric in zip(result, self.model.metrics_names):
      print("{0}: {1:.3e}".format(metric, res))

The model evaluation occurs using both the training data and the training label data. The evaluation for each metric used is then printed in scientific notation.

#### Prediction method

In [ ]:
def predict(self):
    print("predict")

    y_reshaped, y_real = None, self.processor.rescale(self.prediction_X[:,4].reshape(len(self.prediction_X), 1), self.prediction_X)

    for hour in range(self.hours):
      self.prediction_X = self.prediction_X[1:] if hour > 1 else self.prediction_X[0:]
      y_output = self.model.predict(self.prediction_X)

      self.prediction_X[:,4] = y_output.reshape(len(y_output))

      y_reshaped = self.processor.rescale(y_output, self.prediction_X)
    
    index_ = self.hours-2 if self.hours > 1 else 0
    rmse = sqrt(mean_squared_error(y_reshaped, y_real[index_:]))
    print('RMSE: %.3f' % rmse)

    y = np.zeros(y_real.shape)
    np.put(y, np.indices(y.shape), np.nan)
    starting_index = len(y) - len(y_reshaped)

    np.put(y, np.indices(y.shape)[:,starting_index:],y_reshaped)

    pyplot.plot(y, label='predicted')
    pyplot.plot(y_real, label='measured')
    pyplot.legend()
    pyplot.show()

A prediction can occur for the next 1, 2 or n hours. The number of hours is specified using the flag -hours (--hours) when executing the main script and then it is saved using the self.hours variable.

Since the model is training considering only the next hours, if a +12 hours prediction is needed, it is necessary to first predict the previous 11 hours (one by one).

Finally, after predicting the last hour, the results are rescaled and then plotted in a graphic.

There were some adjustment to the graphic in order to display all 194 testing labels.

### Complete code

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
from matplotlib import pyplot

from math import sqrt
from sklearn.metrics import mean_squared_error

np.set_printoptions(suppress=True)

DEFAULT_EPOCH = 50
DEFAULT_BATCH = 72

VALIDATION_SPLIT = 0.3
TESTING_SPLIT = 0.3

TOTAL = 744
N_INPUTS = 550

class KerasDenseMLP:

  def __init__(self, processor, args = None, values = []):

    self.processor = processor
    self.values = values

    self.hours = args.hours if args.hours else 1
    self.checkpoint = args.checkpoint if args.checkpoint else "dense_" + str(len(args.neurons)) + "_layers_checkpoint.keras"
    self.epochs = args.epochs if args.epochs else DEFAULT_EPOCH
    self.batch  = args.batch if args.batch else DEFAULT_BATCH

    self.features = args.input

    self.build_data()
    self.model = keras.Sequential()
    
    self.model.add(keras.layers.Dense(
      units=args.neurons[0],
      activation="tanh",
      input_shape=(args.input,)
    ))

    for key, neurons in enumerate(args.neurons):
      if key < len(args.neurons) - 1:
        self.model.add(keras.layers.Dense(units=args.neurons[key+1], activation="tanh"))
      else:
        self.model.add(keras.layers.Dense(units=args.output, activation="tanh"))

    self.model.add(keras.layers.Dense(units=args.output))
    
    print(self.model.summary())
    self.model.compile(
      optimizer=keras.optimizers.SGD(lr=args.learning),
      loss=keras.losses.MSE,
      metrics=[keras.metrics.MSE, keras.metrics.MAE]
    )

  def build_data(self):
    inputs = self.values[0:N_INPUTS]
    prediction = self.values[N_INPUTS:TOTAL]
    
    np.random.shuffle(inputs)
    N_TEST = int(N_INPUTS*TESTING_SPLIT)

    self.train_X, self.train_y = inputs[:(N_INPUTS-N_TEST), :-1], inputs[:(N_INPUTS-N_TEST), -1]
    self.test_X, self.test_y = inputs[(N_INPUTS-N_TEST):, :-1], inputs[(N_INPUTS-N_TEST):, -1]

    self.prediction_X, self.prediction_y = prediction[:, :-1], prediction[:, -1]



  def train(self):
    print("train")
    checkpoint = keras.callbacks.ModelCheckpoint(
      filepath=self.checkpoint,
      monitor="val_loss",
      verbose=1,
      save_weights_only=True,
      save_best_only=True
    )
    early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, verbose=1)
    tensorboard = keras.callbacks.TensorBoard(log_dir="./logs/", histogram_freq=0, write_graph=False)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        min_lr=1e-4,
        patience=0,
        verbose=1
    )
    
    callbacks = [
      checkpoint, early_stopping, tensorboard, reduce_lr
    ]
    
    history = self.model.fit(
      self.train_X, self.train_y, epochs=self.epochs,
      steps_per_epoch=200,
      validation_steps=10,
      validation_split=VALIDATION_SPLIT,
      callbacks=callbacks
    )

    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='validation')
    pyplot.legend()
    pyplot.show()

  def evaluate(self):
    print("eval")
    result = self.model.evaluate(x=self.test_X, y=self.test_y)

    for res, metric in zip(result, self.model.metrics_names):
      print("{0}: {1:.3e}".format(metric, res))

  def predict(self):
    print("predict")

    y_reshaped, y_real = None, self.processor.rescale(self.prediction_X[:,4].reshape(len(self.prediction_X), 1), self.prediction_X)

    for hour in range(self.hours):
      self.prediction_X = self.prediction_X[1:] if hour > 1 else self.prediction_X[0:]
      y_output = self.model.predict(self.prediction_X)

      self.prediction_X[:,4] = y_output.reshape(len(y_output))

      y_reshaped = self.processor.rescale(y_output, self.prediction_X)
    
    index_ = self.hours-2 if self.hours > 1 else 0
    rmse = sqrt(mean_squared_error(y_reshaped, y_real[index_:]))
    print('RMSE: %.3f' % rmse)

    y = np.zeros(y_real.shape)
    np.put(y, np.indices(y.shape), np.nan)
    starting_index = len(y) - len(y_reshaped)

    np.put(y, np.indices(y.shape)[:,starting_index:],y_reshaped)

    pyplot.plot(y, label='predicted')
    pyplot.plot(y_real, label='measured')
    pyplot.legend()
    pyplot.show()